# MPIA Arxiv on Deck 2

Contains the steps to produce the paper extractions.

In [1]:
# Imports
import os
from IPython.display import Markdown, display
from tqdm.notebook import tqdm
import warnings
from PIL import Image 
import re

# requires arxiv_on_deck_2

from arxiv_on_deck_2.arxiv2 import (get_new_papers, 
                                    get_paper_from_identifier,
                                    retrieve_document_source, 
                                    get_markdown_badge)
from arxiv_on_deck_2 import (latex,
                             latex_bib,
                             mpia,
                             highlight_authors_in_list)

# Sometimes images are really big
Image.MAX_IMAGE_PIXELS = 1000000000 

In [2]:
# Some useful definitions.

class AffiliationWarning(UserWarning):
    pass

class AffiliationError(RuntimeError):
    pass

def validation(source: str):
    """Raises error paper during parsing of source file
    
    Allows checks before parsing TeX code.
    
    Raises AffiliationWarning
    """
    check = mpia.affiliation_verifications(source, verbose=True)
    if check is not True:
        raise AffiliationError("mpia.affiliation_verifications: " + check)

        
warnings.simplefilter('always', AffiliationWarning)


def get_markdown_qrcode(paper_id: str):
    """ Generate a qrcode to the arxiv page using qrserver.com
    
    :param paper: Arxiv paper
    :returns: markdown text
    """
    url = r"https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="
    txt = f"""<img src={url}"https://arxiv.org/abs/{paper_id}">"""
    txt = '<div id="qrcode">' + txt + '</div>'
    return txt


def clean_non_western_encoded_characters_commands(text: str) -> str:
    """ Remove non-western encoded characters from a string
    List may need to grow.
    
    :param text: the text to clean
    :return: the cleaned text
    """
    text = re.sub(r"(\\begin{CJK}{UTF8}{gbsn})(.*?)(\\end{CJK})", r"\2", text)
    return text


def get_initials(name: str) -> str:
    """ Get the short name, e.g., A.-B. FamName
    :param name: full name
    :returns: initials
    """
    initials = []
    # account for non western names often in ()
    if '(' in name:
        name = clean_non_western_encoded_characters_commands(name)
        suffix = re.findall(r"\((.*?)\)", name)[0]
        name = name.replace(f"({suffix})", '')
    else:
        suffix = ''
    split = name.split()
    for token in split[:-1]:
        if '-' in token:
            current = '-'.join([k[0] + '.' for k in token.split('-')])
        else:
            current = token[0] + '.'
        initials.append(current)
    initials.append(split[-1].strip())
    if suffix:
        initials.append(f"({suffix})")
    return ' '.join(initials)

## get list of arxiv paper candidates

We use the MPIA mitarbeiter list webpage from mpia.de to get author names
We then get all new papers from Arxiv and match authors

In [3]:
# deal with the author list and edge cases of people that cannot be consistent on their name  

def filter_non_scientists(name: str) -> bool:
    """ Loose filter on expected authorships

    removing IT, administration, technical staff
    :param name: name
    :returns: False if name is not a scientist
    """
    remove_list = ['Licht', 'Binroth', 'Witzel', 'Jordan',
                   'Zähringer', 'Scheerer', 'Hoffmann', 'Düe',
                   'Hellmich', 'Enkler-Scharpegge', 'Witte-Nguy',
                   'Dehen', 'Beckmann', 'Jager', 'Jäger'
                  ]

    for k in remove_list:
        if k in name:
            return False
    return True

def add_author_to_list(author_list: list) -> list:
    """ Add author to list if not already in list
    
    :param author: author name
    :param author_list: list of authors
    :returns: updated list of authors
    """
    add_list = ['T. Henning']

    for author in add_list:
        if author not in author_list:
            author_list.append(author)
    return author_list

# get list from MPIA website
# filter for non-scientists (mpia.get_mpia_mitarbeiter_list() does some filtering)
mpia_authors = [k[1] for k in mpia.get_mpia_mitarbeiter_list() if filter_non_scientists(k[1])]
# add some missing author because of inconsistencies in their MPIA name and author name on papers
mpia_authors = add_author_to_list(mpia_authors)

In [4]:
new_papers = get_new_papers()
# add manual references
add_paper_refs = []
new_papers.extend([get_paper_from_identifier(k) for k in add_paper_refs])

def robust_call(fn, value, *args, **kwargs):
    try:
        return fn(value, *args, **kwargs)
    except Exception:
        return value

candidates = []
for paperk in new_papers:
    # Check author list with their initials
    normed_author_list = [robust_call(mpia.get_initials, k) for k in paperk['authors']]
    hl_authors = highlight_authors_in_list(normed_author_list, mpia_authors, verbose=True)
    matches = [(hl, orig) for hl, orig in zip(hl_authors, paperk['authors']) if 'mark' in hl]
    paperk['authors'] = hl_authors
    if matches:
        # only select paper if an author matched our list
        candidates.append(paperk)
print("""Arxiv has {0:,d} new papers today""".format(len(new_papers)))        
print("""          {0:,d} with possible author matches""".format(len(candidates)))

A. d. Graaff  ->  A. D. Graaff  |  ['A. D. Graaff']
X. Zhang  ->  X. Zhang  |  ['X. Zhang']
N. Wang  ->  N. Wang  |  ['N. Wang']
Y. Wang  ->  Y. Wang  |  ['Y. Wang']


T. Müller  ->  T. Müller  |  ['T. Müller']
H. Beuther  ->  H. Beuther  |  ['H. Beuther']
J. Liu  ->  J. Liu  |  ['J. Liu']
N. Wang  ->  N. Wang  |  ['N. Wang']
K. Jahnke  ->  K. Jahnke  |  ['K. Jahnke']
Arxiv has 124 new papers today
          8 with possible author matches


# Parse sources and generate relevant outputs

From the candidates, we do the following steps:
* get their tarball from ArXiv (and extract data)
* find the main .tex file: find one with \documentclass{...} (sometimes it's non trivial)
* Check affiliations with :func:`validation`, which uses :func:`mpia.affiliation_verifications`
* If passing the affiliations: we parse the .tex source
   * inject sub-documents into the main (flatten the main document)
   * parse structure, extract information (title, abstract, authors, figures...)
   * handles `\graphicspath` if provided
* Generate the .md document.

In [5]:
documents = []
failed = []
for paper in tqdm(candidates):
    # debug crap
    paper['identifier'] = paper['identifier'].lower().replace('arxiv:', '').replace(r'\n', '').strip()
    paper_id = paper['identifier']
    
    folder = f'tmp_{paper_id}'

    try:
        if not os.path.isdir(folder):
            folder = retrieve_document_source(f"{paper_id}", f'tmp_{paper_id}')
        
        try:
            doc = latex.LatexDocument(folder, validation=validation)    
        except AffiliationError as affilerror:
            msg = f"ArXiv:{paper_id:s} is not an MPIA paper... " + str(affilerror)
            failed.append((paper, "affiliation error: " + str(affilerror) ))
            continue
        
        # Hack because sometimes author parsing does not work well
        if (len(doc.authors) != len(paper['authors'])):
            doc._authors = paper['authors']
        else:
            # highlight authors (FIXME: doc.highlight_authors)
            # done on arxiv paper already
            doc._authors = highlight_authors_in_list(
                [get_initials(k) for k in doc.authors], 
                mpia_authors, verbose=True)
        if (doc.abstract) in (None, ''):
            doc._abstract = paper['abstract']
            
        doc.comment = (get_markdown_badge(paper_id) + 
                       "<mark>Appeared on: " + paper['date'] + "</mark> - ")
        if paper['comments']:
            doc.comment += " _" + paper['comments'] + "_"
        
        full_md = doc.generate_markdown_text()
        
        full_md += get_markdown_qrcode(paper_id)
        
        # replace citations
        try:
            bibdata = latex_bib.LatexBib.from_doc(doc)
            full_md = latex_bib.replace_citations(full_md, bibdata)
        except Exception as e:
            print("Issues with the citations")
            print(e)
        
        documents.append((paper_id, full_md))
    except Exception as e:
        warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +
                                         str(e)
                                        ))
        failed.append((paper, "latex error " + str(e)))

  0%|          | 0/8 [00:00<?, ?it/s]

Retrieving document from  https://arxiv.org/e-print/2509.05434


extracting tarball to tmp_2509.05434... done.


/tmp/ipykernel_3396/2822249172.py:52: LatexWarning: 2509.05434 did not run properly
bad escape \i at position 36
  warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +


Retrieving document from  https://arxiv.org/e-print/2509.05705


...

extracting tarball to tmp_2509.06788...

 done.
Retrieving document from  https://arxiv.org/e-print/2509.06805


extracting tarball to tmp_2509.06805...

 done.


/tmp/ipykernel_3396/2822249172.py:52: LatexWarning: 2509.06805 did not run properly
Unable to locate Ghostscript on paths
  warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +


### Export the logs

Throughout, we also keep track of the logs per paper. see `logs-{today date}.md` 

In [6]:
import datetime
today = str(datetime.date.today())
logfile = f"_build/html/logs/log-{today}.md"


with open(logfile, 'w') as logs:
    # Success
    logs.write(f'# Arxiv on Deck 2: Logs - {today}\n\n')
    logs.write("""* Arxiv had {0:,d} new papers\n""".format(len(new_papers)))
    logs.write("""    * {0:,d} with possible author matches\n\n""".format(len(candidates)))
    logs.write("## Sucessful papers\n\n")
    display(Markdown("## Successful papers"))
    success = [k[0] for k in documents]
    for candid in candidates:
        if candid['identifier'].split(':')[-1] in success:
            display(candid)
            logs.write(candid.generate_markdown_text() + '\n\n')

    ## failed
    logs.write("## Failed papers\n\n")
    display(Markdown("## Failed papers"))
    failed = sorted(failed, key=lambda x: x[1])
    current_reason = ""
    for paper, reason in failed:
        if 'affiliation' in reason:
            color = 'green'
        else:
            color = 'red'
        data = Markdown(
                paper.generate_markdown_text() + 
                f'\n|<p style="color:{color:s}"> **ERROR** </p>| <p style="color:{color:s}">{reason:s}</p> |'
               )
        if reason != current_reason:
            logs.write(f'### {reason:s} \n\n')
            current_reason = reason
        logs.write(data.data + '\n\n')
        
        # only display here the important errors (all in logs)
        # if color in ('red',):
        display(data)

## Successful papers

## Failed papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2509.06787-b31b1b.svg)](https://arxiv.org/abs/2509.06787) | **Detection of a septuple stellar system in formation via disk fragmentation**  |
|| S. Li, et al. -- incl., <mark>H. Beuther</mark>, <mark>J. Liu</mark> |
|*Appeared on*| *2025-09-09*|
|*Comments*| *31 pages, 14 figures, 1 table, resubmitted to Nature Astronomy following final referee comments*|
|**Abstract**|            Stellar multiple systems play a pivotal role in cluster dynamics and stellar evolution, leading to intense astronomical phenomena like X-ray binaries, gamma-ray bursts, Type Ia supernova, and stellar mergers, which are prime sources of gravitational waves. However, their origin remains poorly understood. Here we report the discovery of a septuple protostellar system embedded in a Keplerian disk within the high-mass star-forming region NGC\,6334IN, with close separations of 181-461 AU. The stability analysis reveals that the disk surrounding the septuple system is dynamically unstable, indicating that the septuple system formed via disk fragmentation. Previous studies have typically found only 2--3 members forming via disk fragmentation in both low- and high-mass star-forming regions. Our findings provide compelling observational evidence that the fragmentation of a gravitationally unstable disk is a viable mechanism for the formation of extreme high-order multiplicity, confirming what was previously only a theoretical concept. The results shed new light on the formation of extreme high-order multiplicity in cluster environments.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: '69117' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2509.05705-b31b1b.svg)](https://arxiv.org/abs/2509.05705) | **The 21 cm forest one-dimensional power spectrum as a probe of dark matter particles and primordial black holes**  |
|| M.-L. Zhao, Y. Shao, S. Wang, <mark>X. Zhang</mark> |
|*Appeared on*| *2025-09-09*|
|*Comments*| *12 pages, 7 figures*|
|**Abstract**|            Understanding the nature of dark matter (DM) particles remains a pivotal challenge in modern cosmology. Current cosmological research on these phenomena primarily utilizes cosmic microwave background (CMB) observations and other late-time probes, which predominantly focus on large scales. We introduce a novel probe, the 21 cm forest signal, which can be used to investigate DM properties on small scales during the epoch of reionization, thereby addressing the gap left by other cosmological probes. Annihilation and decay of DM particles, as well as Hawking radiation from PBHs, can heat the intergalactic medium (IGM). This heating suppresses the amplitude of the 21 cm forest 1D power spectrum. Therefore, the 1D power spectrum provides an effective method for constraining DM properties. However, astrophysical heating processes in the early universe can also affect the 21 cm forest 1D power spectrum. In this work, we assess the potential of using the SKA to observe the 21 cm forest 1D power spectrum for constraining DM properties, under the assumption that astrophysical heating can be constrained reliably by other independent probes. Under low astrophysical heating conditions, the 1D power spectrum could constrain the DM annihilation cross section and decay lifetime to $\langle\sigma v\rangle \sim {10^{-31}}\,{\rm cm^{3}\,s^{-1}}$ and $\tau \sim {10^{30}}\,{\rm s}$ for ${10}\,{\rm GeV}$ DM particles, and probe PBHs with masses $\sim {10^{15}}\,{\rm\,g}$ at abundances $f_{\mathrm{PBH}} \simeq 10^{-13}$. These constraints represent improvements of 5-6 orders of magnitude over current limits. Furthermore, the 21 cm forest 1D power spectrum has the potential to exceed existing bounds on sub-GeV DM and to probe PBHs with masses above $10^{18}\,{\rm g}$, which are otherwise inaccessible by conventional cosmological probes.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2509.05957-b31b1b.svg)](https://arxiv.org/abs/2509.05957) | **FAST Observations of the Microstructure in Interpulse Pulsars**  |
|| W. Li, et al. -- incl., <mark>N. Wang</mark> |
|*Appeared on*| *2025-09-09*|
|*Comments*| *accepted for publication in ApJS*|
|**Abstract**|            In this study, we investigate the microstructure properties of four pulsars (PSRs J0953+0755 (B0950+08), J0627+0706, J0826+2637 (B0823+26) and J1946+1805 (B1944+17)) using the Five-hundred-meter Aperture Spherical radio Telescope (FAST), with particular emphasis on identifying microstructure within interpulse (IP). Through the application of autocorrelation function (ACF) analysis and fast Fourier transform (FFT) techniques, we have systematically examined the periodicity of microstructure in these pulsars. Our findings represent the first successful detection of microstructure within IP. Furthermore, we conducted a comprehensive statistical analysis comparing the characteristic timescales ($\tau_{\mu}$) and the characteristic periods $P_{\mu}$ of quasi-periodic microstructure between the main pulse (MP) and IP, and our results indicate that the $\tau_{\mu}$ and $P_{\mu}$ of microstructure across components appear consistent within measurement errors for PSR J0627+0706, but microstructure in IP are relatively smaller than those in MP for PSR J0953+0755. Furthermore, the relationship between $P_{\mu}$ of microstructure and the rotation period in neutron star populations was reconfirmed: $P_{\mu}(\text{ms})=(1.337\pm0.114)\times P(\text{s})^{(1.063\pm0.038)}$.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2509.06243-b31b1b.svg)](https://arxiv.org/abs/2509.06243) | **Evidence for Two SNe Type Triggering GRB 220101A: a Pair SN and a Rotating Magnetized Core Collapse SN**  |
|| R. Ruffini, et al. -- incl., <mark>Y. Wang</mark> |
|*Appeared on*| *2025-09-09*|
|*Comments*| *15 pages, 9 figures, submitted to ApJ*|
|**Abstract**|            The traditional model which assumed GRBs as originating from a single Black Hole (BH) have been modified by the Binary Driven Hypernova (BdHN) model in which the trigger of the GRB originate from a binary system composed of a 10 solar mass CO core and a 2 solar mass neutron star binary companion. The binary period being as short as minutes to hours. Recent results evidence that the most energetic GRBs may have as precursors a sequence of two types of SN: a pair-SN followed soon after by a more traditional core collapse. All this appear to open the way to two new types of SNe: the Pair-SNe without a remnant and a magnetized-rotating-core collapse leading to the formation of a pulsar. We generalize our previous assumptions, here the possibility of fission of the rapidly rotating CO core is examined. The role of the magnetic field of the CO core are equally examined with the magneto-hydrodynamic associated process, all the way to generate overcritical fields and e+ e- pairs. All this appear to open the way to a new types of SN, much in line with pair-SN models and a significant change from the initial non rotating models. The BdHN are characterized by seven new physics Episodes here all used to formulate the new approach. The substantial modification of each Episode are outlined by a leading order treatment.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2509.06772-b31b1b.svg)](https://arxiv.org/abs/2509.06772) | **JWST Detection of Hydrocarbon Ices and Methane Gas on Makemake**  |
|| S. Protopapa, et al. -- incl., <mark>T. Müller</mark> |
|*Appeared on*| *2025-09-09*|
|*Comments*| *23 pages, 11 figures, 2 tables. Accepted for publication in The Astrophysical Journal Letters*|
|**Abstract**|            JWST/NIRSpec observations of Makemake reveal a chemically complex surface and evidence of gaseous CH$_4$. Our spectral modeling indicates a surface composition consisting of CH$_4$, CH$_3$D, and possibly CH$_3$OH, combined with aggregates of C$_2$H$_2$ and C$_2$H$_6$. The presence of C$_2$H$_4$ is also considered given its expected photochemical origin. Both areal and layered configurations reproduce the observed spectrum, with the latter being preferred. This composition confirms earlier hydrocarbon detections and suggests that CH$_4$ photolysis is either ongoing or occurred recently. The detection of CH$_3$D yields a D/H ratio in CH$_4$ ice of $(3.98 \pm 0.34) \times 10^{-4}$, consistent within $2\sigma$ with previous estimates. We report the first detection of CH$_4$ fluorescence from Makemake, establishing it as only the second trans-Neptunian object$\unicode{x2014}$after Pluto$\unicode{x2014}$with confirmed volatile release. We explore two scenarios consistent with the observed CH$_4$ emission, though neither fully reproduces the data: an expanding coma, yielding production rates of $(0.2\unicode{x2013}1.6) \times 10^{28}$ molecules s$^{-1}$ and a rovibrational temperature of $\sim$35 K, possibly originating from a localized plume, and a gravitationally bound atmosphere, which, if adopted, implies gas kinetic temperatures near 40 K and surface pressures of $\sim$10 pbar$\unicode{x2014}$values consistent with stellar occultation constraints and an atmosphere in equilibrium with surface CH$_4$ ice. Discriminating between these scenarios will require higher spectral resolution and improved signal-to-noise observations. Together, the gas-phase CH$_4$, intermediate D/H ratio between that in water and CH$_4$ in comets, and complex surface composition challenge the traditional view of Makemake as a quiescent, frozen body.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2509.06788-b31b1b.svg)](https://arxiv.org/abs/2509.06788) | **An Interpretable AI Framework to Disentangle Self-Interacting and Cold Dark Matter in Galaxy Clusters: The CKAN Approach**  |
|| Z. Huang, H. Shi, Z. Liu, <mark>N. Wang</mark> |
|*Appeared on*| *2025-09-09*|
|*Comments*| *15 pages, 8 figures; accepted for publication in The Astronomical Journal*|
|**Abstract**|            Convolutional neural networks have shown their ability to differentiate between self-interacting dark matter (SIDM) and cold dark matter (CDM) on galaxy cluster scales. However, their large parameter counts and ''black-box'' nature make it difficult to assess whether their decisions adhere to physical principles. To address this issue, we have built a Convolutional Kolmogorov-Arnold Network (CKAN) that reduces parameter count and enhances interpretability, and propose a novel analytical framework to understand the network's decision-making process. With this framework, we leverage our network to qualitatively assess the offset between the dark matter distribution center and the galaxy cluster center, as well as the size of heating regions in different models. These findings are consistent with current theoretical predictions and show the reliability and interpretability of our network. By combining network interpretability with unseen test results, we also estimate that for SIDM in galaxy clusters, the minimum cross-section $(\sigma/m)_{\mathrm{th}}$ required to reliably identify its collisional nature falls between $0.1\,\mathrm{cm}^2/\mathrm{g}$ and $0.3\,\mathrm{cm}^2/\mathrm{g}$. Moreover, CKAN maintains robust performance under simulated JWST and Euclid noise, highlighting its promise for application to forthcoming observational surveys.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2509.06805-b31b1b.svg)](https://arxiv.org/abs/2509.06805) | **Euclid preparation. Methodology for validating the Euclid Catalogue of Galaxy Clusters using external data**  |
|| E. Collaboration, et al. -- incl., <mark>K. Jahnke</mark> |
|*Appeared on*| *2025-09-09*|
|*Comments*| *22 pages, 17 figures, submitted to A&A*|
|**Abstract**|            We present our methodology for identifying known clusters as counterparts to objects in the Euclid Catalogue of Galaxy Clusters (ECGC). Euclid is expected to detect a large number of optically-selected galaxy clusters over the approximately 14000 square degrees of its extragalactic sky survey. Extending out well beyond redshift unity, the catalogue will contain many new high-redshift clusters, while at lower redshifts a fraction of the clusters will have been observed in other surveys. Identifying these known clusters as counterparts to the Euclid-detected clusters is an important step in the validation and construction of the ECGC to augment information with external observables. We present a set of catalogues and meta-catalogues of known clusters that we have assembled for this step, and we illustrate their application and our methodology using the Dark Energy Survey Year 1 RedMaPPer cluster catalogue in lieu of the future ECGC. In the process of this work, we have constructed and deliver an updated EC-RedMaPPer catalogue with multi-wavelength counterparts.         |
|<p style="color:red"> **ERROR** </p>| <p style="color:red">latex error Unable to locate Ghostscript on paths</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2509.05434-b31b1b.svg)](https://arxiv.org/abs/2509.05434) | **What you see is what you get: empirically measured bolometric luminosities of Little Red Dots**  |
|| J. E. Greene, et al. -- incl., <mark>A. d. Graaff</mark> |
|*Appeared on*| *2025-09-09*|
|*Comments*| *11 pages, 3 figures, 1 table, posted following referee comments*|
|**Abstract**|            New populations of red active galactic nuclei (known as ``Little Red Dots'') discovered by JWST exhibit remarkable spectral energy distributions. Leveraging X-ray through far-infrared observations of two of the most luminous known Little Red Dots, we directly their bolometric luminosities. We find evidence that more than half of the bolometric luminosity likely emerges in the rest-frame optical, with $L_{\rm bol}/L_{5100} = 5$, roughly half the value for ``standard'' Active Galactic Nuclei. Meanwhile, the X-ray emitting corona, UV-emitting black-body, and reprocessed mid to far-infrared emission are all considerably sub-dominant, assuming that the far-infrared luminosity is well below current measured limits. We present new bolometric corrections that dramatically lower inferred bolometric luminosities by a factor of ten compared to published values in the literature. These bolometric corrections are in accord with expectations from models in which gas absorption and reprocessing are responsible for the red rest-frame optical colors of Little Red Dots. We discuss how this lowered luminosity scale suggests a lower mass scale for the population by at least an order of magnitude {\bf (e.g., $\sim 10^5-10^7~{\rm M_{\odot}}$ black holes, and $\sim 10^8~{\rm M_{\odot}}$ galaxies)}, alleviating tensions with clustering, overmassive black holes, and the integrated black hole mass density in the Universe.         |
|<p style="color:red"> **ERROR** </p>| <p style="color:red">latex error bad escape \i at position 36</p> |

## Export documents

We now write the .md files and export relevant images

In [7]:
def export_markdown_summary(md: str, md_fname:str, directory: str):
    """Export MD document and associated relevant images"""
    import os
    import shutil
    import re

    if (os.path.exists(directory) and not os.path.isdir(directory)):
        raise RuntimeError(f"a non-directory file exists with name {directory:s}")

    if (not os.path.exists(directory)):
        print(f"creating directory {directory:s}")
        os.mkdir(directory)

    fig_fnames = (re.compile(r'\[Fig.*\]\((.*)\)').findall(md) + 
                  re.compile(r'\<img src="([^>\s]*)"[^>]*/>').findall(md))
    print("found figures", fig_fnames)
    for fname in fig_fnames:
        if 'http' in fname:
            # No need to copy online figures
            continue
        if not os.path.exists(fname):
            print("file not found", fname)
            continue
        print("copying ", fname, "to", directory)
        destdir = os.path.join(directory, os.path.dirname(fname))
        destfname = os.path.join(destdir, os.path.basename(fname))
        try:
            os.makedirs(destdir)
        except FileExistsError:
            pass
        shutil.copy(fname, destfname)
    with open(os.path.join(directory, md_fname), 'w') as fout:
        fout.write(md)
    print("exported in ", os.path.join(directory, md_fname))
    [print("    + " + os.path.join(directory,fk)) for fk in fig_fnames]

In [8]:
for paper_id, md in documents:
    export_markdown_summary(md, f"{paper_id:s}.md", '_build/html/')

## Display the papers

Not necessary but allows for a quick check.

In [9]:
[display(Markdown(k[1])) for k in documents];

# Create HTML index

In [10]:
from datetime import datetime, timedelta, timezone
from glob import glob
import os

files = glob('_build/html/*.md')
days = 7
now = datetime.today()
res = []
for fk in files:
    stat_result = os.stat(fk).st_ctime
    modified = datetime.fromtimestamp(stat_result, tz=timezone.utc).replace(tzinfo=None)
    delta = now.today() - modified
    if delta <= timedelta(days=days):
        res.append((delta.seconds, fk))
res = [k[1] for k in reversed(sorted(res, key=lambda x:x[1]))]
npub = len(res)
print(len(res), f" publications files modified in the last {days:d} days.")
# [ print('\t', k) for k in res ];

128  publications files modified in the last 7 days.


In [11]:
import datetime
from glob import glob

def get_last_n_days(lst, days=1):
    """ Get the documents from the last n days """
    sorted_lst = sorted(lst, key=lambda x: x[1], reverse=True)
    for fname, date in sorted_lst:
        if date >= str(datetime.date.today() - datetime.timedelta(days=days)):
            yield fname

def extract_appearance_dates(lst_file):
    dates = []

    def get_date(line):
        return line\
            .split('Appeared on:')[-1]\
            .split('</mark>')[0].strip()

    for fname in lst:
        with open(fname, 'r') as f:
            found_date = False
            for line in f:
                if not found_date:
                    if "Appeared on" in line:
                        found_date = True
                        dates.append((fname, get_date(line)))
                else:
                    break
    return dates

from glob import glob
lst = glob('_build/html/*md')
days = 7
dates = extract_appearance_dates(lst)
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last {days:d} days.")

10  publications in the last 7 days.


In [12]:
def create_carousel(npub=4):
    """ Generate the HTML code for a carousel with `npub` slides """
    carousel = ["""  <div class="carousel" """,
                """       data-flickity='{ "autoPlay": 10000, "adaptiveHeight": true, "resize": true, "wrapAround": true, "pauseAutoPlayOnHover": true, "groupCells": 1 }' id="asyncTypeset">"""
                ]
    
    item_str = """    <div class="carousel-cell"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        carousel.append(item_str.format(k=k))
    carousel.append("  </div>")
    return '\n'.join(carousel)

def create_grid(npub=4):
    """ Generate the HTML code for a flat grid with `npub` slides """
    grid = ["""  <div class="grid"> """,
                ]
    
    item_str = """    <div class="grid-item"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        grid.append(item_str.format(k=k))
    grid.append("  </div>")
    return '\n'.join(grid)

In [13]:
carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "7-day archives" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
with open("_build/html/index_7days.html", 'w') as fout:
    fout.write(page)

In [14]:
# redo for today
days = 1
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last day.")

carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "Daily" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(carousel, docs, slides)
# print(page)
with open("_build/html/index_daily.html", 'w') as fout:
    fout.write(page)

2  publications in the last day.


In [15]:
# Create the flat grid of the last N papers (fixed number regardless of dates)
from itertools import islice 

npub = 6
res = [k[0] for k in (islice(reversed(sorted(dates, key=lambda x: x[1])), 6))]
print(len(res), f" {npub} publications selected.")

grid = create_grid(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("grid_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- grid-content:s --%}", grid)\
               .replace("{%-- suptitle:s --%}",  f"Last {npub:,d} papers" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(grid, docs, slides)
# print(page)
with open("_build/html/index_npub_grid.html", 'w') as fout:
    fout.write(page)

6  6 publications selected.
